In [1]:
import os
from pprint import pprint
from tqdm import tqdm
import  spacy
from spacy.tokens import DocBin

In [2]:
from easynmt import EasyNMT
model = EasyNMT('opus-mt')

In [73]:
sentence = "Он положил свою руку ей на плечо"
model.translate(sentence, source_lang='ru', target_lang='en', beam_size=10, max_length=200)

c:\users\warri\pycharmprojects\translational_variability\venv\lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


'He put his hand on her shoulder.'

In [74]:
with open("corpora/subtitles/ru_subtitles_spacy_dump.bin", "rb") as f:
    restored_bytes_data = f.read()

nlp = spacy.blank("ru")
doc_bin = DocBin().from_bytes(restored_bytes_data)
ru_all_docs = list(doc_bin.get_docs(nlp.vocab))
ru_all_docs[0]

Дети могут достать во дворе почти всё что угодно до тех пор , пока могут себе это позволить . 

In [83]:
with open("subtitles_raw/ru-en_0-900.txt", "rt", encoding="utf-8") as f:
    text = f.read()

data = text.split('================================')

In [84]:
data = [item.strip() for item in data]

ru = []
en = []
idx = 0
for item in data:
    if item.startswith('#') or not item:
        continue
    sentences = item.split('\n')
    src = []
    trg = []
    for sentence in sentences:
        # print(idx, sentence)
        if sentence.startswith("(src)"):
            trash, useful = sentence.split(">")
            useful = useful.split()
            src += useful
        else:
            trash, useful = sentence.split(">")
            useful = useful.split()
            trg += useful
        idx += 1

    ru.append(src)
    en.append(trg)

In [86]:
with open("subtitles_raw/ru_raw_0-900.txt", "wt", encoding="utf-8") as f:
    for sentence in ru:
        string = ' '.join(sentence)
        f.write(string)
        f.write('\n')

In [87]:
with open("subtitles_raw/en_raw_0-900.txt", "wt", encoding="utf-8") as f:
    for sentence in en:
        string = ' '.join(sentence)
        f.write(string)
        f.write('\n')

считаем индексы где встретилось искомое слово

In [78]:
def find_lines_with_word(original_word: str, docs) -> list:
    indexes = []
    for i, sentence in enumerate(docs):
        for token in sentence:
            if token.lemma_.lower() == original_word:
                indexes.append(i)
                break
    return indexes

In [79]:
original_word = "имя"
indexes = find_lines_with_word(original_word, ru_all_docs)
len(indexes)

2443

In [80]:
chunk_size = 1000
latest_chunk = 0
for chunk_start in range(latest_chunk, len(indexes), chunk_size):
    translations_text = ""

    for i in tqdm(range(chunk_start, min(chunk_start + chunk_size, len(indexes)))):
        sentence = ' '.join(ru[indexes[i]])
        translation = model.translate(sentence, source_lang='ru', target_lang='en', beam_size=10, max_length=200)
        translations_text += translation
        translations_text += '\n'

    folder_name = "corpora/subtitles/translations/name"
    filename = f"{folder_name}/opus10_{chunk_start}-{chunk_start + chunk_size - 1}"

    with open(filename, 'wt', encoding="utf-8") as f:
        f.write(translations_text)

  0%|          | 0/1000 [00:01<?, ?it/s]


KeyboardInterrupt: 

выравнивание перевода

In [3]:
from simalign import SentenceAligner
from spacy.tokens import DocBin

In [4]:
myaligner = SentenceAligner(model="bert", token_type="bpe", matching_methods="mai")
en_nlp_lg = spacy.load("en_core_web_lg")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-05-09 16:47:41,303 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: bert-base-multilingual-cased


In [88]:
with open("corpora/subtitles/translations/name/opus10.txt", "rt", encoding="utf-8") as f:
    translated_word_corpus_raw = [line.strip() for line in f]

translated_word_corpus_raw[:3]

['- Name?', 'Name?', 'Name?']

In [9]:
whole_corpus = ["" for _ in range(900000)]

In [110]:
for i in range(len(indexes)):
    whole_corpus[indexes[i]] = translated_word_corpus_raw[i]

In [10]:
with open("corpora/subtitles/translations/opus10_whole.txt", "wt+", encoding="utf-8") as f:
    for line in whole_corpus:
        f.write(line)
        f.write('\n')

In [97]:
with open("corpora/subtitles/translations/name/opus10_spacy.bin", "rb") as file:
    translated_bytes_data = file.read()

nlp = spacy.blank("en")
doc_bin = DocBin().from_bytes(translated_bytes_data)
translated_docs = list(doc_bin.get_docs(nlp.vocab))
translated_docs[5]

Your name.

In [5]:
en_nlp_md = spacy.load("en_core_web_md")

In [7]:
empty = en_nlp_lg("")
docs = [empty for _ in range(900000)]

In [102]:
doc_bin = DocBin()
for doc in docs:
    doc_bin.add(doc)
bytes_data = doc_bin.to_bytes()

with open("corpora/subtitles/translations/opus10_spacy_whole.txt", "wb") as file:
    file.write(bytes_data)

In [ ]:
docs = []
for sentence in tqdm(translated_word_corpus_raw):
    spacy_doc = en_nlp_lg(sentence)
    docs.append(spacy_doc)

doc_bin = DocBin()
for doc in docs:
    doc_bin.add(doc)
bytes_data = doc_bin.to_bytes()

with open("corpora/subtitles/translations/name/opus10_spacy.txt", "wb") as file:
    file.write(bytes_data)

In [ ]:
mwmf = ""
itermax = ""
inter = ""

for i in tqdm(range(len(indexes))):

    translated_tokens = [token.lemma_ for token in docs[i]]
    src, trg = (ru[indexes[i]], translated_tokens)
    # print(f"{ru[indexes[i]]}\n{translated_tokens}\n")

    alignments = myaligner.get_word_aligns(src, trg)
    mwmf += " ".join([f"{x}-{y}" for x, y in alignments["mwmf"]])
    itermax += " ".join([f"{x}-{y}" for x, y in alignments["itermax"]])
    inter += " ".join([f"{x}-{y}" for x, y in alignments["inter"]])
    mwmf += "\n"
    itermax += "\n"
    inter += "\n"

folder_name = "corpora/subtitles/translations/name/numerical"
fname_mwmf = f"{folder_name}/mwmf"
fname_itermax = f"{folder_name}/itermax"
fname_inter = f"{folder_name}/inter"

with open(fname_mwmf, 'wt', encoding="utf-8") as f:
    f.write(mwmf)
with open(fname_itermax, 'wt', encoding="utf-8") as f:
    f.write(itermax)
with open(fname_inter, 'wt', encoding="utf-8") as f:
    f.write(inter)

устраиваем хранение в общем файле